In [1]:
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
import argparse
import torch
# from pypai.model import upload_model
import time
import multiprocessing
import pandas as pd
import os
# from odps import DataFrame
# from odps import options
# from pypai.io import TableWriter
# from pypai.io import TableReader
# from pypai.utils import env_utils
# from aistudio_common.utils import oss_utils

import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn.functional import edge_softmax

Using backend: pytorch


In [43]:
import pandas as pd
path='data/DBLP/'

labels = pd.read_csv(path+"label.dat", sep = "\t", header=None,
                 names=["user_id","user_name","user_type","label"])

labels_test = pd.read_csv(path+"label.dat.test", sep = "\t", header=None,
                 names=["user_id","user_name","user_type","label"])

# 4种类型的节点统一编号
nodes = pd.read_csv(path+"node.dat", sep = "\t", header=None,
                 names=["new_id","node_name","node_type","feats"])

# 6种类型的边[0-5]
links = pd.read_csv(path+"link.dat", sep = "\t", header=None,
                 names=["new1_id","new2_id","link_type","feats"])

# 数据说明
metas = pd.read_csv(path+"meta.dat", sep = "\t", header=None)

In [19]:
# names=["new_id","node_name","node_type","feats","rename_id"]
type0_nodes = nodes[nodes['node_type'] == 0]
type0_nodes['rename_id'] = range(len(type0_nodes))

type1_nodes = nodes[nodes['node_type'] == 1]
type1_nodes['rename_id'] = range(len(type1_nodes))

type2_nodes = nodes[nodes['node_type'] == 2]
type2_nodes['rename_id'] = range(len(type2_nodes))

type3_nodes = nodes[nodes['node_type'] == 3]
type3_nodes['rename_id'] = range(len(type3_nodes))

edge_type0_id1 = links[links['link_type'] == 0][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type0_id2 = links[links['link_type'] == 0][['new2_id']].rename(columns={'new2_id':'new_id'})

edge_type1_id1 = links[links['link_type'] == 1][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type1_id2 = links[links['link_type'] == 1][['new2_id']].rename(columns={'new2_id':'new_id'})

edge_type2_id1 = links[links['link_type'] == 2][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type2_id2 = links[links['link_type'] == 2][['new2_id']].rename(columns={'new2_id':'new_id'})

edge_type3_id1 = links[links['link_type'] == 3][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type3_id2 = links[links['link_type'] == 3][['new2_id']].rename(columns={'new2_id':'new_id'})

edge_type4_id1 = links[links['link_type'] == 4][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type4_id2 = links[links['link_type'] == 4][['new2_id']].rename(columns={'new2_id':'new_id'})

edge_type5_id1 = links[links['link_type'] == 5][['new1_id']].rename(columns={'new1_id':'new_id'})
edge_type5_id2 = links[links['link_type'] == 5][['new2_id']].rename(columns={'new2_id':'new_id'})


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [20]:
edge_type0_id1_rename = pd.merge(edge_type0_id1,type0_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=0
edge_type0_id2_rename = pd.merge(edge_type0_id2,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1

edge_type1_id1_rename = pd.merge(edge_type1_id1,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1
edge_type1_id2_rename = pd.merge(edge_type1_id2,type2_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=2

edge_type2_id1_rename = pd.merge(edge_type2_id1,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1
edge_type2_id2_rename = pd.merge(edge_type2_id2,type3_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=3

edge_type3_id1_rename = pd.merge(edge_type3_id1,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1
edge_type3_id2_rename = pd.merge(edge_type3_id2,type0_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=0

edge_type4_id1_rename = pd.merge(edge_type4_id1,type2_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=2
edge_type4_id2_rename = pd.merge(edge_type4_id2,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1

edge_type5_id1_rename = pd.merge(edge_type5_id1,type3_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=3
edge_type5_id2_rename = pd.merge(edge_type5_id2,type1_nodes,how='left',on='new_id')['rename_id'].tolist()  # node_type=1


In [21]:
def do_eval(feat):
    return np.array(eval(feat))


def eval_features(feats):
    p = multiprocessing.Pool()
    results = p.map(do_eval, feats)
    p.close()
    p.join()
    return results


def preprocess_feature(feats):
    lengths = len(feats)
    print('>>>>>>',lengths)
    sep = 5000000
    trans_num = max(1, 1+int(lengths/sep))
    for i in range(trans_num):
        if i==0:
            if lengths>sep:
                result = eval_features(feats[i*sep:i*sep+sep])
            else:
                result = eval_features(feats)
            out = torch.tensor(result).float()
        elif i!=trans_num-1:
            result = torch.tensor(eval_features(feats[i*sep:i*sep+sep])).float()
            out = torch.cat([out, result], dim=0)
        else:
            result = torch.tensor(eval_features(feats[i*sep:])).float()
            out = torch.cat([out, result], dim=0)
    return out

In [53]:
labels = labels.rename(columns={'user_id':'new_id'})
labels['is_label'] = [1]*len(labels)
type0_nodes_labels = pd.merge(type0_nodes,labels,how='left',on='new_id').sort_values('rename_id', ascending=True)

type0_nodes_labels['label'] = type0_nodes_labels[['label']].fillna(0)
type0_nodes_labels['is_label'] = type0_nodes_labels[['is_label']].fillna(0)

labels_test = labels_test.rename(columns={'user_id':'new_id','label':'label_test'})
type0_nodes_labels = pd.merge(type0_nodes_labels,labels_test,how='left',on='new_id').sort_values('rename_id', ascending=True)
type0_nodes_labels['label_test'] = type0_nodes_labels[['label_test']].fillna(0)
type0_nodes_labels['label'] = type0_nodes_labels['label']+type0_nodes_labels['label_test']

tmp = pd.merge(labels_test,type0_nodes,how='left',on='new_id')
test_idx = tmp['rename_id'].tolist()

In [54]:
labels_test

new_id           user_name  user_type  label_test
0       2211         Neal Palmer          0           2
1       1806        Jianhua Feng          0           2
2        532           Jiajun Bu          0           0
3        852  Stephen J. Buckley          0           0
4       3366       Shengnan Cong          0           0
...      ...                 ...        ...         ...
2835    3827          Duygu Ucar          0           3
2836    3762      Dave Northcutt          0           2
2837    1475           Wenjie Li          0           3
2838     871       Paul D. Hubbe          0           0
2839    2467  Ravi Krishnamurthy          0           0

[2840 rows x 4 columns]

In [58]:
a = [1,2,3,4,5]
np.random.RandomState(1).shuffle(a)
a

[3, 2, 5, 1, 4]

In [8]:
print('building hetero-graph')
g = dgl.heterograph({
    ('t0', 'l0', 't1'): (edge_type0_id1_rename, edge_type0_id2_rename),
    ('t1', 'l1', 't2'): (edge_type1_id1_rename, edge_type1_id2_rename),
    ('t1', 'l2', 't3'): (edge_type2_id1_rename, edge_type2_id2_rename),
    ('t1', 'l3', 't0'): (edge_type3_id1_rename, edge_type3_id2_rename),
    ('t2', 'l4', 't1'): (edge_type4_id1_rename, edge_type4_id2_rename),
    ('t3', 'l5', 't1'): (edge_type5_id1_rename, edge_type5_id2_rename)
}, num_nodes_dict={'t0': len(type0_nodes), 't1': len(type1_nodes), 't2': len(type2_nodes), 't3': len(type3_nodes)}
)
print(g)
# g.nodes['A'].data['del_t'] = torch.tensor(node_data1['del_t'].tolist()).long()
# g.nodes['B'].data['del_t'] = torch.tensor(node_data2['del_t'].tolist()).long()
## 随便生成时间，实际应该从odps表里面读
# g.nodes['A'].data['del_t'] = torch.tensor(np.zeros(len(node_data1), dtype=np.int))
# g.nodes['B'].data['del_t'] = torch.tensor(np.zeros(len(node_data2), dtype=np.int))

# print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
g.nodes['t0'].data['feat'] = preprocess_feature(type0_nodes['feats'].values)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
g.nodes['t1'].data['feat'] = preprocess_feature(type1_nodes['feats'].values)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
g.nodes['t2'].data['feat'] = preprocess_feature(type2_nodes['feats'].values)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

g.nodes['t3'].data['feat'] = torch.tensor([[0]]*len(type3_nodes))

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

g.nodes['t0'].data['label'] = torch.tensor(type0_nodes_labels['label'].astype('int').tolist())

g.nodes['t0'].data['is_label'] = torch.tensor(type0_nodes_labels['is_label'].astype('int').tolist())
g.nodes['t1'].data['is_label'] = torch.tensor([[0]]*len(type1_nodes))
g.nodes['t2'].data['is_label'] = torch.tensor([[0]]*len(type2_nodes))
g.nodes['t3'].data['is_label'] = torch.tensor([[0]]*len(type3_nodes))

# labels = g.nodes['t0'].data['label']
t0_labels = g.nodes['t0'].data['label']

node_dict = {}
edge_dict = {}

for ntype in g.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in g.etypes:
    edge_dict[etype] = len(edge_dict)
    g.edges[etype].data['id'] = torch.ones(g.number_of_edges(etype), dtype=torch.long) * edge_dict[etype]
    
# n_inp_dict - 
    

building hetero-graph
Graph(num_nodes={'t0': 4057, 't1': 14328, 't2': 7723, 't3': 20},
      num_edges={('t0', 'l0', 't1'): 19645, ('t1', 'l1', 't2'): 85810, ('t1', 'l2', 't3'): 14328, ('t1', 'l3', 't0'): 19645, ('t2', 'l4', 't1'): 85810, ('t3', 'l5', 't1'): 14328},
      metagraph=[('t0', 't1', 'l0'), ('t1', 't2', 'l1'), ('t1', 't3', 'l2'), ('t1', 't0', 'l3'), ('t2', 't1', 'l4'), ('t3', 't1', 'l5')])
>>>>>> 4057
2021-12-22 07:35:18
>>>>>> 14328
2021-12-22 07:36:41
>>>>>> 7723
2021-12-22 07:36:44
2021-12-22 07:36:44


In [9]:
# generate train/val/test split
pid = len(type0_nodes)
shuffle = np.random.RandomState(11).permutation(pid) #固定打乱顺序
# pid = len(shuffle)
val_idx = torch.tensor(shuffle[0:int(pid * 0.2)]).long()
train_idx = torch.tensor(shuffle[int(pid * 0.2):]).long() 

print('number of validation data:', len(val_idx))
print('number of positive labels:', torch.sum(t0_labels[val_idx]==1))
print('number of negative labels:', torch.sum(t0_labels[val_idx]==0))

number of validation data: 811
number of positive labels: tensor(48)
number of negative labels: tensor(77)


In [10]:
## 参数设置

device = torch.device("cuda:0")
# device = torch.device('cpu')

parser = argparse.ArgumentParser(description='Training GNN on ogbn-products benchmark')

parser.add_argument('--n_epoch', type=int, default=300)
parser.add_argument('--n_hid', type=int, default=64)
# parser.add_argument('--n_inp', type=int, default=g.nodes['N'].data['feat'].shape[-1])  
parser.add_argument('--clip', type=int, default=1.0)

parser.add_argument('--max_lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=float, default=1024)
parser.add_argument('--max_steps', type=int, default=1000)
parser.add_argument('--val_step', type=int, default=1)


# args = parser.parse_args()
args = parser.parse_args(args=[])

node_inp_dict = dict()
for key in node_dict:
    node_inp_dict[node_dict[key]] = g.nodes[key].data['feat'].shape[-1]

In [11]:
## 定义各种方法、模型，可以跳过
### train和inference两个函数对同质图进行了改写

class HGTLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 node_dict,
                 edge_dict,
                 n_heads,
                 dropout = 0.2,
                 use_norm = False,
                 out_key='',
                 is_change_fe = False):
        super(HGTLayer, self).__init__()

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.node_dict     = node_dict
        self.edge_dict     = edge_dict
        self.num_types     = len(node_dict)
        self.num_relations = len(edge_dict)
        self.total_rel     = self.num_types * self.num_relations * self.num_types
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None
        self.out_key = out_key

        self.k_linears   = nn.ModuleList()
        self.q_linears   = nn.ModuleList()
        self.v_linears   = nn.ModuleList()
        self.a_linears   = nn.ModuleList()
        self.norms       = nn.ModuleList()
        self.use_norm    = use_norm

        for t in range(self.num_types):
            self.k_linears.append(nn.Linear(in_dim,   out_dim))
            self.q_linears.append(nn.Linear(in_dim,   out_dim))
            self.v_linears.append(nn.Linear(in_dim,   out_dim))
            self.a_linears.append(nn.Linear(out_dim,  out_dim))
            if use_norm:
                self.norms.append(nn.LayerNorm(out_dim))

        self.relation_pri   = nn.Parameter(torch.ones(self.num_relations, self.n_heads))
        self.relation_att   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.skip           = nn.Parameter(torch.ones(self.num_types))
        self.drop           = nn.Dropout(dropout)
        self.my_alpha = torch.tensor(0.)
        
        self.is_change_fe = is_change_fe

        nn.init.xavier_uniform_(self.relation_att)
        nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, G, h):
        with G.local_scope():
            node_dict, edge_dict = self.node_dict, self.edge_dict
            for srctype, etype, dsttype in G.canonical_etypes:
                sub_graph = G[srctype, etype, dsttype]
                
                k_linear = self.k_linears[node_dict[srctype]]
                v_linear = self.v_linears[node_dict[srctype]]
                q_linear = self.q_linears[node_dict[dsttype]]

                k = k_linear(h[srctype]).view(-1, self.n_heads, self.d_k)  ## 得到多头的key, query, value, size: (node_nums, n_heads, out_dim // n_heads)
                v = v_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                q = q_linear(h[dsttype][:G.number_of_dst_nodes(dsttype)]).view(-1, self.n_heads, self.d_k)

                e_id = self.edge_dict[etype]

                relation_att = self.relation_att[e_id]  ## size: (n_heads, out_dim // n_heads, out_dim // n_heads)
                relation_pri = self.relation_pri[e_id]
                relation_msg = self.relation_msg[e_id]
                
#                 print('relation_att',relation_att.shape)

                k = torch.einsum("bij,ijk->bik", k, relation_att)
                v = torch.einsum("bij,ijk->bik", v, relation_msg)

                sub_graph.srcdata['k'] = k
                sub_graph.dstdata['q'] = q
                sub_graph.srcdata['v_%d' % e_id] = v

                sub_graph.apply_edges(fn.v_dot_u('q', 'k', 't'))  # 对q和k特征进行点乘，求kq值，准备求attention value
                attn_score = sub_graph.edata.pop('t').sum(-1) * relation_pri / self.sqrt_dk
                attn_score = edge_softmax(sub_graph, attn_score, norm_by='dst')

                sub_graph.edata['t'] = attn_score.unsqueeze(-1)   ## 得到每一种类型的attention value
            ## message passing along the eage type
            G.multi_update_all({etype : (fn.u_mul_e('v_%d' % e_id, 't', 'm'), fn.sum('m', 't')) \
                                for etype, e_id in edge_dict.items()}, cross_reducer = 'mean')
            new_h = {}
            '''
                Step 3: Target-specific Aggregation
                x = norm( W[node_type] * gelu( Agg(x) ) + x )
            '''
            
            for ntype in G.ntypes:
                n_id = node_dict[ntype]
                alpha = torch.sigmoid(self.skip[n_id])
                # print(alpha)
                t = G.dstnodes[ntype].data['t'].view(-1, self.out_dim)
                
                
                if self.is_change_fe:
                    dst_nodes = h[ntype][:G.number_of_dst_nodes(ntype)]
                    is_label = G.dstnodes[ntype].data['is_label'].view(-1,1)
#                     print('>>>>>>>>>>>>>>>>>is_label: ',is_label.shape)
#                     print('>>>>>>>>>>>>>>>>>h[ntype][:G.number_of_dst_nodes(ntype)]: ',h[ntype][:G.number_of_dst_nodes(ntype)].shape)
                    label_nodes = is_label*dst_nodes
                    unlabel_nodes = (1-is_label)*dst_nodes
                    unlabel_t = (1-is_label)*t
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                    
                    unlabel_idx = (is_label == 0).nonzero()[:,0]
#                     print('is_label: ',is_label)
#                     print('unlabel_idx',unlabel_idx)
                    
                    delta = torch.abs(unlabel_t[unlabel_idx] - unlabel_nodes[unlabel_idx])
                    sorted_values,sorted_indexs = torch.sort(delta, dim=-1) #按行排序，sorted_values[:,0]得到邻居节点最相似的特征
            
                    dst_fe = torch.gather(unlabel_nodes[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
                    t_fe = torch.gather(t[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
#                     print('dst_fe: ',dst_fe)
#                     print('t_fe: ',t_fe)

                    # 替换目标节点的特征
                    # dim=1表示逐行进行列填充
                    unlabel_nodes[unlabel_idx] = unlabel_nodes[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), t_fe.view(-1,1))
                    dst_nodes = label_nodes + unlabel_nodes
                    
                    # 替换邻居节点的特征
                    unlabel_t[unlabel_idx] = unlabel_t[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), dst_fe.view(-1,1))
                    t = is_label*t + unlabel_t
                    trans_out = self.drop(self.a_linears[n_id](t))
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                
                    trans_out = trans_out * alpha + dst_nodes * (1-alpha)
                
                else:
                    trans_out = self.drop(self.a_linears[n_id](t))

                    trans_out = trans_out * alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-alpha)
                # trans_out = trans_out * self.my_alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-self.my_alpha)
                if self.use_norm:
                    new_h[ntype] = self.norms[n_id](trans_out)
                else:
                    new_h[ntype] = trans_out
                
            return new_h

class HGT(nn.Module):
    def __init__(self, G, node_dict, edge_dict, node_inp_dict, n_hid, n_out, n_layers, n_heads, use_norm = True, out_key='', use_t=False):
        super(HGT, self).__init__()
        self.node_dict = node_dict
        self.edge_dict = edge_dict
        
        ##kk
        self.gcs = nn.ModuleList()
        self.gcs2 = nn.ModuleList()
        ##kk
        
        self.node_inp_dict = node_inp_dict
        self.n_hid = n_hid
        self.n_out = n_out
        self.n_layers = n_layers
        self.adapt_ws  = nn.ModuleList()
        for t in range(len(node_dict)):
#             print('test1>>>>>>>>> t: ',t)
            self.adapt_ws.append(nn.Linear(self.node_inp_dict[t], n_hid))
        
        ##kk
        for _ in range(n_layers):
            self.gcs.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, use_norm = use_norm, out_key=out_key))
            self.gcs2.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, use_norm = use_norm, out_key=out_key, is_change_fe=True))
        ##kk
        
        #self._in = nn.Linear(n_inp,   n_hid)
        self.out = nn.Linear(n_hid, n_out)
        self.log_softmax_out = nn.LogSoftmax(dim=1)
        self.softmax_out = nn.Softmax(dim=1)
        self.use_t = use_t
#         self.RTE = RelTemporalEncoding(n_inp, max_len=367)

    def forward(self, block, out_key):
        h = {}
        h2 = {}
        for b in range(len(block)):
            if b == 0:
                h = {}
                h2 = {}
                for ntype in block[b].ntypes:   ## 这里写入模型输入
                    n_id = self.node_dict[ntype]
                    if self.use_t:
                        inp_emd = self.RTE(block[b].srcnodes[ntype].data['feat'], block[b].srcnodes[ntype].data['del_t'])
                        h[ntype] = F.gelu(self.adapt_ws[n_id](inp_emd))
                        h2[ntype] = F.gelu(self.adapt_ws[n_id](inp_emd))
                    else:
                        try:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat']))
                            h2[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat']))
                        except:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat'].float()))
                            h2[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat'].float()))
#             print('>>>>>>>>>>>>>>>>>h: ',h['_N'].shape)
#             print('>>>>>>>>>>>>>>>>>h2: ',h2['_N'].shape)
            h = self.gcs[b](block[b], h) ## 表示有多少层HGT，层数越多，跳数越多
            h2 = self.gcs2[b](block[b], h2)
        return self.softmax_out(self.out(h[out_key])), self.softmax_out(self.out(h2[out_key]))


class RelTemporalEncoding(nn.Module):
    '''
        Implement the Temporal Encoding (Sinusoid) function.
    '''
    def __init__(self, n_hid, max_len = 367, dropout = 0.2):
        super(RelTemporalEncoding, self).__init__()
        position = torch.arange(0., max_len, 1).unsqueeze(1)
        div_term = (torch.exp(torch.arange(0, n_hid, 2) *
                             -(math.log(10000.0) / n_hid)))

        emb = nn.Embedding(max_len, n_hid)
        emb.weight.data[:, 0::2] = torch.sin(position * div_term) / math.sqrt(n_hid)
        emb.weight.data[:, 1::2] = torch.cos(position * div_term) / math.sqrt(n_hid)
        emb.requires_grad = False
        self.emb = emb
        self.lin = nn.Linear(n_hid, n_hid)

    def forward(self, x, t):
        try:
            self.emb(t)
        except:
            print(t)
        return x + self.lin(self.emb(t))   ## 注意两者的embedding维度要一样
    

def do_eval(feat):
    return np.array(eval(feat))


def eval_features(feats):
    p = multiprocessing.Pool()
    results = p.map(do_eval, feats)
    p.close()
    p.join()
    return results


def preprocess_feature(feats):
    lengths = len(feats)
    sep = 5000000
    trans_num = max(1, 1+int(lengths/sep))
    for i in range(trans_num):
        if i==0:
            if lengths>sep:
                result = eval_features(feats[i*sep:i*sep+sep])
            else:
                result = eval_features(feats)
            out = torch.tensor(result).float()
        elif i!=trans_num-1:
            result = torch.tensor(eval_features(feats[i*sep:i*sep+sep])).float()
            out = torch.cat([out, result], dim=0)
        else:
            result = torch.tensor(eval_features(feats[i*sep:])).float()
            out = torch.cat([out, result], dim=0)
    return out


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp


def build_dataloader(G, sample_list, dst_dict, batch_size):
    sampler = dgl.dataloading.neighbor.MultiLayerNeighborSampler(sample_list)
    collator = dgl.dataloading.NodeCollator(G, dst_dict, sampler)
    dataloader = torch.utils.data.DataLoader(collator.dataset, collate_fn=collator.collate, batch_size=batch_size, shuffle=True, drop_last=False)
    return dataloader


def train(model, G, dst_key, sample_list, train_dst_dict, val_dst_dict, batch_size, save_name, model_i):
    best_val_acc = torch.tensor(0).cuda()
    best_test_acc = torch.tensor(0)
    # train_ = torch.tensor(0)
    train_step = 0
    for epoch in np.arange(args.n_epoch) + 1:
        print('>>>>>>>>>>>> epoch >>>>>>>>>>>>>',epoch)
        
        dataloader = build_dataloader(G, sample_list, train_dst_dict, batch_size)
        
        for input_nodes, output_nodes, blocks in dataloader:
            blocks = [b.to(device) for b in blocks]
            print('blocks: ',len(dataloader))
            if train_step > args.max_steps-1:
                break
            model.train()
            
            ##kk
            logits1,logits2 = model(blocks, dst_key)
            logits1 = torch.log(logits1)
            logits2 = torch.log(logits2)
            ##kk
            
            # loss = F.cross_entropy(logits, blocks[-1].dstdata['label'][dst_key]) 
            # loss = F.nll_loss(logits, blocks[-1].dstdata['label'][dst_key])  ## 绑定输入为log 概率
            
            ##kk
            is_label = blocks[-1].dstdata['is_label'][dst_key]  ##新增一列标记数据是否为有标签数据
            labels_idx = (is_label == 1).nonzero().view(-1,1)  
            unlabels_idx = (is_label == 0).nonzero().view(-1,1)
#             print('is_label: ',is_label)
#             print('******************is_label: ',is_label.shape)
#             print('******************logits1: ',logits1.shape)
#             print('******************logits1: ',logits1)
#             print('******************blocks[-1].dstdata[\'label\']: ',blocks[-1].dstdata['label'][dst_key].shape)
#             print('******************(1-is_label): ',(1-is_label).shape)
#             print('******************logits2: ',logits2.shape)
#             print('*****************(is_label*logits1).squeeze', torch.squeeze(is_label*logits1,1).shape)
#             print('labels_idx: ',labels_idx.shape,labels_idx)
#             print('unlabels_idx: ',unlabels_idx.shape,unlabels_idx)
#             print('logits1[labels_idx]: ',logits1[labels_idx].shape)
#             print('blocks[-1].dstdata: ',torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1))
#             print('logits2[unlabels_idx]: ',logits2[unlabels_idx].shape)
#             print('logits1: ',len(logits1))
#             print('logits2: ',len(logits2))
            ## 绑定输入为log 概率,同质图用这一行
#             loss = F.nll_loss(torch.squeeze(is_label*logits1,1), torch.squeeze(is_label*blocks[-1].dstdata['label'].view(-1,1),1)) + F.mse_loss(torch.squeeze((1-is_label)*logits1,1), torch.squeeze((1-is_label)*logits2,1))
            if not len(labels_idx):
                loss = F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1))
                print('#################nll_loss: None ')
                print('#################mse_loss: ',F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1)))
            if not len(unlabels_idx):
                loss = F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1))
                print('#################nll_loss: ',F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)))
                print('#################mse_loss: None ')
            ## 异质图
            if len(labels_idx) and len(unlabels_idx):
                rate = 0.2
                print('#################nll_loss: ',F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)))
                print('#################mse_loss: ',F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1)))
                loss = (1-rate)*F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)) + rate*F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1))
            ##kk

            
            # The loss is computed only for labeled nodes.
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()
            train_step += 1

            if train_step % args.val_step == 0:
                if train_step!=1:
                    scheduler.step()
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
                print("train_step", train_step)
                train_pred = logits1.argmax(1)
#                 print('train_pred: ',train_pred[labels_idx])
#                 print('blocks[-1]: ',blocks[-1].dstdata['label'][dst_key][labels_idx])
                
                train_acc = (train_pred[labels_idx] == blocks[-1].dstdata['label'][dst_key][labels_idx]).float().mean()
                                
                # train_acc = (train_pred == blocks[-1].dstdata['label']).float().mean() # 同质图用这一行
        
                
                val_acc_list = []
                model.eval()
                dataloader_val = build_dataloader(G, sample_list, val_dst_dict, batch_size)
                
                for input_nodes0, output_nodes0, blocks0 in dataloader_val: 
                    
                    is_val_label = blocks0[-1].dstdata['is_label'][dst_key]  ##新增一列标记数据是否为有标签数据
                    val_labels_idx = (is_val_label == 1).nonzero().view(-1,1)
                    if not len(val_labels_idx):
                        continue
                    
                    blocks0 = [b.to(device) for b in blocks0]
                    logits1,_ = model(blocks0, dst_key)  ## 做inference
                    pred = logits1.argmax(1)
#                     print('pred[val_labels_idx]: ',pred[val_labels_idx])
#                     print('label: ',blocks0[-1].dstdata['label'][dst_key][val_labels_idx])
#                     print('result: ',pred[val_labels_idx] == blocks0[-1].dstdata['label'][dst_key][val_labels_idx])
                    
                    val_acc = (pred[val_labels_idx] == blocks0[-1].dstdata['label'][dst_key][val_labels_idx]).float().mean()
                    # val_acc = (pred == blocks0[-1].dstdata['label']).float().mean().cuda() # 同质图用这一行
                    val_acc_list.append(val_acc)           
                    
                
                val_acc = sum(val_acc_list)/len(val_acc_list)
#                 print('val_acc in device :',val_acc_list)
#                 print('best_val_acc in device :',len(val_acc_list).device)
                
                if best_val_acc < val_acc and val_acc-best_val_acc>0.0005:
                    best_val_acc = val_acc
                    if train_step>int(0.01*args.max_steps):
                        torch.save(model.state_dict(), 'best_' + save_name+'_'+str(model_i)+'.pth')
                        # model_id = upload_model('best_' + save_name+".pth", name="HGT_model", model_oss_url='oss://cmps-model/323064/GNN/best_'+save_name+'.pth')
                        # model_id = upload_model('best_' + save_name+".pth", name="HGT_model")

                        print('save best model in step ', train_step)
                
                print('step: %d LR: %.5f train_Loss %.4f, train_acc %.4f, Val Acc %.4f (Best %.4f)' % (
                    train_step,
                    optimizer.param_groups[0]['lr'],
                    loss.item(),
                    train_acc.item(),
                    val_acc.item(),
                    best_val_acc.item(),
                ))


def inference(model, G, dst_key, sample_list, dst_dict, batch_size, write_odps_table_name, raw_label='', project='ant_prot_dev'):
    print("strat to inference")
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    model.eval()
    dataloader = build_dataloader(G, sample_list, dst_dict, batch_size)  # 此处batch_size可以调大些加速inference
    infer_step = 0
    for input_nodes, output_nodes, blocks in dataloader: 
        blocks = [b.to(device) for b in blocks]
        logits,_ = model(blocks, dst_key)  # 做inference, 不需要再求log
        if raw_label!='':
            label = G.nodes[dst_key].data[raw_label][output_nodes[dst_key]].reshape([-1, 1])
            # label = G.nodes[dst_key].data[raw_label][output_nodes].reshape([-1, 1]) # 同质图用这一行
            
        else:
            label = torch.tensor([-1]*len(logits)).reshape([-1, 1])
        # print(label)

        nodes = output_nodes[dst_key].reshape([-1, 1])
        # nodes = output_nodes.reshape([-1, 1]) # 同质图这一行
        nodes = nodes.cuda()
        label = label.cuda()
        logits = logits.cuda()
        
        # alipay_id = blocks[-1].dstnodes['A'].data['alipay_id'].reshape([-1, 1]).cuda()
        if infer_step==0:
            infer_data = torch.cat([nodes, label, logits], dim=1).cuda()
            
#             infer_data = torch.cat([nodes, label, logits[:,:2]], dim=1).cuda()
#             label_data = torch.cat(label).cuda()
#             pred_data = torch.cat(logits[:,:2].argmax(1)).cuda()
        else:
            infer_data0 = torch.cat([nodes, label, logits], dim=1).cuda()
            infer_data = torch.cat([infer_data, infer_data0], dim=0)
#             infer_data0 = torch.cat([nodes, label, logits[:,:2]], dim=1).cuda()
#             infer_data = torch.cat([infer_data, infer_data0], dim=0)
#             label_data = torch.cat([label_data,label],dim=0)
#             pred_data = torch.cat(pred_data,[logits[:,:2].argmax(1)],dim=0)
        infer_step+=1

        if infer_step%50==0:
            print("infer_step:", infer_step)
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


    infer_data = infer_data.cpu().detach().numpy()
    # writer = TableWriter(table=project+'.'+write_odps_table_name, drop_if_exists=True)   ## 写到odps表中
    ## writer = TableWriter(table=project+'.'+write_odps_table_name, drop_if_exists=True, partition="dt='20210808'")
    columns = ['id', 'label']+['score'+str(i) for i in range(len(logits[0]))]
    pd_df = pd.DataFrame(infer_data, columns=columns, dtype=float)
#     pd_df[['id','label','score1']].to_csv('pred_df.csv')
    # writer.to_table(pd_df)
    pred_data = pd_df.drop(['id', 'label'], axis=1).values.argmax(1)
    print('label>>>>>',len(pd_df[['label']].values))
    print('logits>>>>>',len(pred_data))
#     print('***********',pd_df)
#     print('***********',np.array(pd_df[['label']].values == pred_data))
#     pred_acc = np.mean(pd_df[['label']].values == pred_data)
    
#     pred_acc = np.mean(pd_df[['label']].values == pred_data)
#     print('pred_acc: ',pred_acc)
    print('inference succeed')
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    return pd_df

In [ ]:
# 训练模型
# model = HGT(g,
#             node_dict, edge_dict,
#             node_inp_dict=node_inp_dict,
#             n_hid=args.n_hid,
#             n_out=t0_labels.max().item() + 1,
#             n_layers=3,
#             n_heads=8,
#             use_norm=True,
#             out_key='t0',
#             use_t=False).to(device)

print(args.n_epoch)
# print(args.n_hid)

# optimizer = torch.optim.AdamW(model.parameters())
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=int(1+args.max_steps/args.val_step), max_lr=args.max_lr, div_factor=25.)
# print('Training HGT with #param: %d' % (get_n_params(model)))
sample_list = [{
#                 ('t0', 'l0', 't1'): 10,
#                 ('t1', 'l1', 't2'): 10,
#                 ('t1', 'l2', 't3'): 10,
#                 ('t1', 'l3', 't0'): 10,
#                 ('t2', 'l4', 't1'): 10,
#                 ('t3', 'l5', 't1'): 10
                ('t0', 'l0', 't1'): 30,
                ('t1', 'l1', 't2'): 30,
                ('t1', 'l2', 't3'): 30,
                ('t1', 'l3', 't0'): 30,
                ('t2', 'l4', 't1'): 30,
                ('t3', 'l5', 't1'): 30
                }, 
               
               {('t0', 'l0', 't1'): 30,
                ('t1', 'l1', 't2'): 30,
                ('t1', 'l2', 't3'): 30,
                ('t1', 'l3', 't0'): 30,
                ('t2', 'l4', 't1'): 30,
                ('t3', 'l5', 't1'): 30}, ] # 两层模型，需要定义每一层聚合neighbor的数量
train_dst_dict = {'t0': train_idx}
val_dst_dict = {'t0': val_idx}
save_name = 'HGT_model_221530_dblp_semi'
# train(model, g, 't0', sample_list, train_dst_dict, val_dst_dict, args.batch_size, save_name)
# torch.save(model.state_dict(), save_name+'.pth')
# model_id = upload_model(save_name+".pth", name="HGT_model")

for model_i in range(5):
    model = HGT(g,
            node_dict, edge_dict,
            node_inp_dict=node_inp_dict,
            n_hid=args.n_hid,
            n_out=t0_labels.max().item() + 1,
            n_layers=2,
            n_heads=8,
            use_norm=True,
            out_key='t0',
            use_t=False).to(device)
    optimizer = torch.optim.AdamW(model.parameters())
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=int(1+args.max_steps/args.val_step), max_lr=args.max_lr, div_factor=25.)
    print('****************************************************')
    print('Training HGT with #param: %d' % (get_n_params(model)))
    train(model, g, 't0', sample_list, train_dst_dict, val_dst_dict, args.batch_size, save_name, model_i)
    torch.save(model.state_dict(), save_name+'_'+str(model_i)+'.pth')
# model_id = upload_model(save_name+".pth", name="HGT_model")

300
****************************************************
Training HGT with #param: 589012
>>>>>>>>>>>> epoch >>>>>>>>>>>>> 1
blocks:  4
#################nll_loss:  tensor(1.6581, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.9808, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-22 07:56:53
train_step 1
step: 1 LR: 0.00004 train_Loss 1.5227, train_acc 0.2007, Val Acc 0.1875 (Best 0.1875)
blocks:  4
#################nll_loss:  tensor(1.6458, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.9785, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-22 07:56:53
train_step 2
step: 2 LR: 0.00004 train_Loss 1.5123, train_acc 0.2079, Val Acc 0.1875 (Best 0.1875)
blocks:  4
#################nll_loss:  tensor(1.6561, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.9609, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-22 07:56:54
train_step 3
step: 3 LR: 0.00004 train_Loss 1.5171, train_acc 

In [17]:
# tmp = type0_nodes_labels.loc[type0_nodes_labels['rename_id'].isin(list(shuffle[int(pid * 0.85):]))]
# test = tmp.loc[tmp['is_label'].isin([1])]
# new_test_idx = test['rename_id'].tolist()

In [27]:
from sklearn.metrics import f1_score
import numpy as np
# 做inference
# sample_list = [{('B', 'r1', 'A'): 10, ('A', 'r2', 'A'): 10, ('A', 'r3', 'B'): 10}, {('B', 'r1', 'A'): 10, ('A', 'r2', 'A'): 10, ('A', 'r3', 'B'): 10}]
model = HGT(g,
            node_dict, edge_dict,
            node_inp_dict=node_inp_dict,
            n_hid=args.n_hid,
            n_out=t0_labels.max().item() + 1,
            n_layers=2,
            n_heads=8,
            use_norm=True,
            out_key='N',
            use_t=False).to(device)

sample_list = [{('t0', 'l0', 't1'): 30,
                ('t1', 'l1', 't2'): 30,
                ('t1', 'l2', 't3'): 30,
                ('t1', 'l3', 't0'): 30,
                ('t2', 'l4', 't1'): 30,
                ('t3', 'l5', 't1'): 30}, 
               
               {('t0', 'l0', 't1'): 30,
                ('t1', 'l1', 't2'): 30,
                ('t1', 'l2', 't3'): 30,
                ('t1', 'l3', 't0'): 30,
                ('t2', 'l4', 't1'): 30,
                ('t3', 'l5', 't1'): 30}, ] # 两层模型，需要定义每一层聚合neighbor的数量

save_name = 'HGT_model_221530_dblp_semi'
dst_dict = {"t0": torch.tensor(test_idx)}
# model.load_state_dict(torch.load('best_' + save_name+'.pth'))
# # model.load_state_dict(torch.load(save_name+'.pth'))
# # write_odps_table_name = 'ant_prot_dev_minghai_323064_inference_test'

# pd_df = inference(model, g, "t0", sample_list, dst_dict, args.batch_size, '', raw_label='label', project='ant_prot_dev')

### 多分类，只能评估acc ###
# pred = pd_df.sort_values(by='score1',ascending=False)
# pred['pred_label'] = [1]*46+[0]*46

# print('pred: ',pd_df[['label']].values.reshape(-1).astype('int32').tolist())
# print('label: ',blocks0[-1].dstdata['label'][dst_key][val_labels_idx])
# print('result: ',pred[val_labels_idx] == blocks0[-1].dstdata['label'][dst_key][val_labels_idx])


# pred_acc = np.mean(pd_df[['label']].values.reshape(-1).astype('int32').tolist() == pd_df.drop(['id','label'], axis=1).values.argmax(1))
# print('pred_acc: ',pred_acc)
# recall = sum(pred['label'][:47])/sum(pred['label'])
# print('recall: ',recall)
# precesion = sum(pred['label'][:47])/46
# print('precesion: ',precesion)

macro = []
micro = []
for model_i in range(10):
    model.load_state_dict(torch.load('best_' + save_name+'_'+str(model_i)+'.pth'))
    # model.load_state_dict(torch.load(save_name+'.pth'))
    # write_odps_table_name = 'ant_prot_dev_minghai_323064_inference_test'

    pd_df = inference(model, g, "t0", sample_list, dst_dict, args.batch_size, '', raw_label='label', project='ant_prot_dev')

    ### 多分类，只能评估acc ###
    # pred = pd_df.sort_values(by='score1',ascending=False)
    # pred['pred_label'] = [1]*46+[0]*46

#     pred_acc = np.mean(pd_df[['label']].values.reshape(-1).astype('int32').tolist() == pd_df.drop(['id','label'], axis=1).values.argmax(1))
#     print('pred_acc: ',pred_acc)
    # recall = sum(pred['label'][:47])/sum(pred['label'])
    # print('recall: ',recall)
    # precesion = sum(pred['label'][:47])/46
    # print('precesion: ',precesion)
    pred = pd_df.drop(['id','label'], axis=1).values.argmax(1).astype('int32')
    label = pd_df[['label']].values.reshape(-1).astype('int32')
    macro_f1 = f1_score(label, pred, average='macro')
    micro_f1 = f1_score(label, pred, average='micro')
    print(str(model_i)+'>>>> macro_f1 >>>>>>>>>> ',macro_f1)
    print(str(model_i)+'>>>> micro_f1 >>>>>>>>>> ',micro_f1)
    macro.append(macro_f1)
    micro.append(micro_f1)
    
m_macro = np.mean(macro)
std_macro = np.std(macro)
m_micro = np.mean(micro)
std_micro = np.std(micro)

print('final result macro_f1 >>>>>>>>>> ',m_macro,'+-',1.96*std_macro)
print('final result micro_f1 >>>>>>>>>> ',m_micro,'+-',1.96*std_micro)

strat to inference
2021-12-17 05:50:18
label>>>>> 184
logits>>>>> 184
inference succeed
2021-12-17 05:50:18
0>>>> macro_f1 >>>>>>>>>>  0.7992650792288012
0>>>> micro_f1 >>>>>>>>>>  0.8043478260869565
strat to inference
2021-12-17 05:50:18
label>>>>> 184
logits>>>>> 184
inference succeed
2021-12-17 05:50:19
1>>>> macro_f1 >>>>>>>>>>  0.8388958854944067
1>>>> micro_f1 >>>>>>>>>>  0.8423913043478262
strat to inference
2021-12-17 05:50:19
label>>>>> 184
logits>>>>> 184
inference succeed
2021-12-17 05:50:19
2>>>> macro_f1 >>>>>>>>>>  0.7606446132483868
2>>>> micro_f1 >>>>>>>>>>  0.7717391304347826
strat to inference
2021-12-17 05:50:19
label>>>>> 184
logits>>>>> 184
inference succeed
2021-12-17 05:50:19
3>>>> macro_f1 >>>>>>>>>>  0.7681133514466849
3>>>> micro_f1 >>>>>>>>>>  0.782608695652174
strat to inference
2021-12-17 05:50:19
label>>>>> 184
logits>>>>> 184
inference succeed
2021-12-17 05:50:19
4>>>> macro_f1 >>>>>>>>>>  0.784116093847747
4>>>> micro_f1 >>>>>>>>>>  0.7934782608695652
st